In [1]:
# Создаём спарк контекст
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [2]:
spark

In [3]:
train = sc.textFile('test.csv')
test = sc.textFile('test.csv')

In [4]:
train_first_row = train.first()

train = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [5]:
train_first_row

'id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

In [8]:
test_first_row = test.first()

test = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [9]:
test_first_row

'id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

In [14]:
combined = train.union(test)

In [15]:
combined.take(5)

[[1.0,
  1043.0,
  1.0,
  1.8,
  1.0,
  14.0,
  0.0,
  5.0,
  0.1,
  193.0,
  3.0,
  16.0,
  226.0,
  1412.0,
  3476.0,
  12.0,
  7.0,
  2.0,
  0.0,
  1.0,
  0.0],
 [2.0,
  841.0,
  1.0,
  0.5,
  1.0,
  4.0,
  1.0,
  61.0,
  0.8,
  191.0,
  5.0,
  12.0,
  746.0,
  857.0,
  3895.0,
  6.0,
  0.0,
  7.0,
  1.0,
  0.0,
  0.0],
 [3.0,
  1807.0,
  1.0,
  2.8,
  0.0,
  1.0,
  0.0,
  27.0,
  0.9,
  186.0,
  3.0,
  4.0,
  1270.0,
  1366.0,
  2396.0,
  17.0,
  10.0,
  10.0,
  0.0,
  1.0,
  1.0],
 [4.0,
  1546.0,
  0.0,
  0.5,
  1.0,
  18.0,
  1.0,
  25.0,
  0.5,
  96.0,
  8.0,
  20.0,
  295.0,
  1752.0,
  3893.0,
  10.0,
  0.0,
  7.0,
  1.0,
  1.0,
  0.0],
 [5.0,
  1434.0,
  0.0,
  1.4,
  0.0,
  11.0,
  1.0,
  49.0,
  0.5,
  108.0,
  6.0,
  18.0,
  749.0,
  810.0,
  1773.0,
  15.0,
  8.0,
  7.0,
  1.0,
  0.0,
  1.0]]

In [17]:
def ratio(row):
    px_width, px_height = row[14], row[13]
    return px_width / px_height

aspect_ratio = combined.map(lambda row: row + [ratio(row)])

wide_screen = aspect_ratio.map(lambda row: row + [1 if row[-1] >= 16/9 else 0])

sorted_distribution = wide_screen \
    .map(lambda row: (row[-1], 1)) \
    .reduceByKey(lambda x, y: x + y)

if sorted_distribution.isEmpty():
    print("RDD пуст. Нет данных для отображения.")
else:
    print("Отсортированное распределение:")
    for item in sorted_distribution.take(10):
        print(f"Экран {'широкоформатный' if item[0] == 1 else 'не широкоформатный'}: {item[1]}")


Отсортированное распределение:
Экран не широкоформатный: 1016
Экран широкоформатный: 984


In [19]:
def ddiagonal(row):
    px_width, px_height = row[14], row[13]  # Индексы столбцов с шириной и высотой
    inches = (px_width**2 + px_height**2)**0.5 / 100  # Переводим из пикселей в дюймы
    return inches

diagonal = combined.map(lambda row: row + [ddiagonal(row)])


In [21]:
def ppi_func(row):
    px_width, px_height, inches = row[14], row[13], row[-1]
    resolution = (px_width**2 + px_height**2)**0.5
    ppi = resolution / inches
    return ppi

ppi = diagonal.map(lambda row: row + [ppi_func(row)])

sorted_ppi_distribution = ppi \
    .map(lambda row: (row[-1], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .sortByKey() \
    .collect()

# Проверяем, что RDD не пуст
if not sorted_ppi_distribution:
    print("RDD пуст. Нет данных для отображения.")
else:
    # Выводим первые несколько элементов
    print("Отсортированное распределение плотности пикселей (ppi):")
    for item in sorted_ppi_distribution[:10]:
        print(f"ppi: {item[0]}, Количество устройств: {item[1]}")



Отсортированное распределение плотности пикселей (ppi):
ppi: 99.99999999999999, Количество устройств: 150
ppi: 100.0, Количество устройств: 1710
ppi: 100.00000000000001, Количество устройств: 140


In [34]:
px_height_index = 12
px_width_index = 13

train2 = train \
    .map(lambda row: (float(row[px_height_index]), float(row[px_width_index])))

test2 = test \
    .map(lambda row: (float(row[px_height_index]), float(row[px_width_index])))

In [35]:
test2.take(5)

[(226.0, 1412.0),
 (746.0, 857.0),
 (1270.0, 1366.0),
 (295.0, 1752.0),
 (749.0, 810.0)]

In [36]:
from math import sqrt

def diagonal(row):
    px_width, px_height = row
    inches = sqrt(px_width**2 + px_height**2) / 100
    return inches

train_diagonal = train2.map(lambda row: (row, diagonal(row)))
test_diagonal = test2.map(lambda row: (row, diagonal(row)))

result = train_diagonal.join(test_diagonal)

In [37]:
result_csv = result.map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]))
result_csv \
    .map(lambda row: ';'.join(map(str, row))) \
    .saveAsTextFile('result.csv')